In [1]:
#import sys
#sys.path.append('../')
#from src.dataloader import EcomedDataset

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import os
import rasterio
import matplotlib.pyplot as plt

In [16]:
#img name img_zone1_0_0_patch_1_12.tif
# msk name msk_zone1_0_0_patch_1_12.tif

class EcomedDataset(Dataset):
    def __init__(self, img_dir, msk_dir, level):
        self.img_dir = img_dir
        self.msk_dir = msk_dir
        self.level = level
        self.imgs = list(img_dir.rglob('*.tif'))
        self.msks = self.imgs.copy()
        for i, img_path in enumerate(self.imgs):
            # get the before last part of the path
            zone = img_path.parts[-2] 
            img_name = img_path.name
            msk_name = img_name.replace('img', 'msk')
            msk_path = msk_dir / zone / msk_name
            self.msks[i] = msk_path
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        msk_path = self.msks[idx]
        with rasterio.open(img_path) as src:
            img = src.read()
        with rasterio.open(msk_path) as src:
            msk = src.read(self.level)
        img = img.astype(np.int16)
        return img, msk

In [17]:
#
def plot_img_msk(img, msk):
    # the 3 bands from img should be rescaled between p2 and p98
    p2, p98 = np.percentile(img, (2, 98))
    img = np.clip(img, p2, p98)
    fig, ax = plt.subplots(1, 2, figsize=(10, 7))
    ax[0].imshow(img.transpose(1, 2, 0))
    ax[1].imshow(msk)
    plt.show()

In [18]:
settings_dict = {
    'patch_size': 256, 
    'level': 1, 
    'bs': 32,#32 
    'shuffle': 'random', 
}
img_path = Path(f'../data/patch{settings_dict["patch_size"]}/img/')
msk_path = Path(f'../data/patch{settings_dict["patch_size"]}/msk/l123/')

In [19]:
# Create a dataloader for level 1
dataset = EcomedDataset(img_path, msk_path, settings_dict['level'])
dataloader = DataLoader(dataset, batch_size=settings_dict['bs'], shuffle=True)

In [28]:
#load data
for i, data in enumerate(dataloader):
    img, msk = data
    print(img.shape, msk.shape)
    print('hey')
    #plot
    plot_img_msk(img[0], msk[0])
    break

RasterioIOError: ../data/patch256/msk/l123/zone16_0_0/msk_zone16_0_0_patch_14_25.tif: No such file or directory